Run pip install if there is no uproot package:
```bash
pip install --upgrade uproot4 awkward1 particle boost-histogram matplotlib mplhep pandas numexpr autograd hist hist[plot] humanize
```

In [12]:
#!pip install --upgrade --user uproot4 awkward1 particle boost-histogram matplotlib mplhep pandas numexpr autograd hist hist[plot] humanize

In [13]:
import uproot4
from matplotlib import pyplot as p
from hist import Hist
import hist
import awkward1 as ak
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg
from pprint import pprint

# Pretty printing arrays
from pprint import pprint

In [14]:


file=uproot4.open("g4e_k_lambda_18on275_10k_l2.run.root")

# To see all object names in the file:
file.items()

# only names: file.keys(), file.values()))

# To see items in subdirectories
# file.keys(recursive=True)

# There are more functions related to file:
# https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyFile.html

# Root directories:
# https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html

# To read multiple files at once (aka TChain)
# https://uproot.readthedocs.io/en/latest/basic.html#iterating-over-many-files


[('events;2', <TTree 'events' (76 branches) at 0x01e345210cc8>),
 ('events;1', <TTree 'events' (76 branches) at 0x01e349444d48>)]

In [15]:
# File has a data tree called 'events'
# get events tree as a separate thing
tree = file["events"]

# Print number of entries
print(f"Number of entries: {tree.num_entries}")

# Uncomment this to see all values:
tree.show()


# More TBranch functions:
#https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html


Number of entries: 1000
name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
event_id             | uint64_t                 | AsDtype('>u8')
evt_true_q2          | double                   | AsDtype('>f8')
evt_true_x           | double                   | AsDtype('>f8')
evt_true_y           | double                   | AsDtype('>f8')
evt_true_w2          | double                   | AsDtype('>f8')
evt_true_nu          | double                   | AsDtype('>f8')
evt_true_t_hat       | double                   | AsDtype('>f8')
evt_has_dis_info     | int8_t                   | AsDtype('int8')
evt_weight           | double                   | AsDtype('>f8')
hit_count            | uint64_t                 | AsDtype('>u8')
hit_id               | std::vector<uint64_t>    | AsJagged(AsDtype('>u8'), he...
hit_trk_id           | std::vector<uint64_t>    | AsJagged(AsDtype('>u8'), he...
h

```
EVENTS
+---------------------+
| event_id            |
| evt_true_q2         |
| evt_true_x          |
| evt_true_y          |
| evt_true_w2         |
| evt_true_nu         |
| evt_true_t_hat      |
| evt_has_dis_info    |
| evt_weight          |
+---------------------+

GENERATED PARTICLES
Particles that where generated by knowing the theory

+---------------------+
|gen_prt_count        |
|gen_prt_id           |      # Unique number/id of this particle inside the event
|gen_prt_vtx_id       |
|gen_prt_pdg          |      # Type of the particle
|gen_prt_trk_id       |
|gen_prt_charge       |
|gen_prt_dir_x        |
|gen_prt_dir_y        |
|gen_prt_dir_z        |
|gen_prt_tot_mom      |
|gen_prt_tot_e        |      # Particle totoal momentum
|gen_prt_time         |
|gen_prt_polariz_x    |
|gen_prt_polariz_y    |
|gen_prt_polariz_z    |
|gen_vtx_count        |
|gen_vtx_id           |
|gen_vtx_part_count   |
|gen_vtx_x            |
|gen_vtx_y            |
|gen_vtx_z            |
|gen_vtx_time         |
|gen_vtx_weight       |
+---------------------+


TRACKS - Simulated particle tracks in our detector
+---------------------+
|trk_count            |         # A number of tracks in an event
|trk_id               | -+-+    # Unique number/id of this particle inside the event
|trk_pdg              |  | |    # Type of the particle
|trk_parent_id        | -+ |    # Parent id of the particle
|trk_create_proc      |    |
|trk_level            |    |
|trk_vtx_x            |    |
|trk_vtx_y            |    |
|trk_vtx_z            |    |
|trk_vtx_dir_x        |    |    # Track direction x,y,z
|trk_vtx_dir_y        |    |
|trk_vtx_dir_z        |    |
|trk_mom              |    |    # Track momentum
+---------------------+    |
HITS in the detector       |
+---------------------+    |
|hit_count            |    |    # Number of hits in the event
|hit_id               |    |
|hit_trk_id           | ---+
|hit_ptr_id           |
|hit_parent_trk_id    |
|hit_vol_name         |         # Volume name where hit occured
|hit_x                |
|hit_y                |
|hit_z                |
|hit_i_rep            |
|hit_j_rep            |
|hit_e_loss           |
+---------------------+


What was in reality:

event 1
    track 1 - electron 10GeV
        hit_11
        hit_12
        hit_13
    track 2 - proton   100GeV
        hit_21
        hit_22


How we write this data:

event_id = 1
trk_count = 2
trk_id = [1, 2]
trk_pdg = [11, 2112]
trk_mom = [10, 100]
hit_count = 5
hit_...=[...] - array of 5 elements
hit_id = [1, 2, 3, 4, 5]
hit_trk_id = [1, 1, 1, 2, 2]
hit_z = [10, 100, 200, -1, -5]
```

In [32]:
trk_pdg = tree['trk_pdg'].array()
trk_vtx_dir_x = tree['trk_vtx_dir_x'].array()
trk_vtx_dir_y = tree['trk_vtx_dir_y'].array()
trk_vtx_dir_z = tree['trk_vtx_dir_z'].array()
trk_mom = tree['trk_mom'].array()

In [25]:
filter_proton = trk_pdg == 2212
filter_pion = trk_pdg == -211

In [28]:
proton_px = trk_vtx_dir_x[filter_proton] * trk_mom[filter_proton]
proton_py = trk_vtx_dir_y[filter_proton] * trk_mom[filter_proton]
proton_pz = trk_vtx_dir_z[filter_proton] * trk_mom[filter_proton]

pion_px = trk_vtx_dir_x[filter_pion] * trk_mom[filter_pion]
pion_py = trk_vtx_dir_y[filter_pion] * trk_mom[filter_pion]
pion_pz = trk_vtx_dir_z[filter_pion] * trk_mom[filter_pion]

In [36]:
pprint(ak.to_list(proton_px[:10]))

pprint(ak.to_list(pion_px[:10]))

[[],
 [],
 [12.153922745551096],
 [11.663091079967003],
 [10.476887109589718],
 [11.053234258968946],
 [],
 [12.271986027398508],
 [],
 [11.861161974561853]]
[[],
 [],
 [1.3018169919531395],
 [1.076818608535191],
 [],
 [1.9024354552344793],
 [],
 [1.3135937078002549],
 [],
 [1.728818025343182]]


In [57]:
pairs = ak.cartesian([proton_px[:10], ak.to_list(pion_px[:10])])

unpaired = ak.unzip(pairs)

In [58]:
pprint(ak.to_list(unpaired[0]))


[[],
 [],
 [12.153922745551096],
 [11.663091079967003],
 [],
 [11.053234258968946],
 [],
 [12.271986027398508],
 [],
 [11.861161974561853]]


In [45]:
proton_energy = np.sqrt(proton_px*proton_px + proton_py*proton_py + proton_pz*proton_pz)
pion_energy = np.sqrt(pion_px*pion_px + pion_py*pion_py + pion_pz*pion_pz)

In [46]:
lam_energy2 = (proton_energy + pion_energy)**2
lam_energy2

ValueError: in ListOffsetArray64, cannot broadcast nested list

(https://github.com/scikit-hep/awkward-1.0/blob/1.0.2/src/cpu-kernels/awkward_ListArray_broadcast_tooffsets.cpp#L27)